In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [86]:
#read in action matrix data
am = pd.read_excel("AM_2006_2019.xlsx")

In [87]:
am.head()

,Unit,Column,Notes,Quarter,Year,Quarter.1,Region,Column #
0,Arkansas Nuclear 1,Licensee Response Column,Baseline Inspection,2006Q1,2006,Q1,Region 4,1
1,Arkansas Nuclear 1,Licensee Response Column,Baseline Inspection,2006Q2,2006,Q2,Region 4,1
2,Arkansas Nuclear 1,Licensee Response Column,Baseline Inspection,2006Q3,2006,Q3,Region 4,1
3,Arkansas Nuclear 1,Licensee Response Column,Baseline Inspection,2006Q4,2006,Q4,Region 4,1
4,Arkansas Nuclear 1,Licensee Response Column,Baseline Inspection,2007Q1,2007,Q1,Region 4,1


In [88]:
#simplify AM data to necessary columns
am = am[["Unit","Quarter","Year","Column #"]]

In [89]:
am.head()

,Unit,Quarter,Year,Column #
0,Arkansas Nuclear 1,2006Q1,2006,1
1,Arkansas Nuclear 1,2006Q2,2006,1
2,Arkansas Nuclear 1,2006Q3,2006,1
3,Arkansas Nuclear 1,2006Q4,2006,1
4,Arkansas Nuclear 1,2007Q1,2007,1


In [90]:
#read in spreadsheet matching units to standardized site names
convert = pd.read_excel("plantsToSites.xlsx")

In [91]:
convert.head()

,Plant,Site
0,Arkansas Nuclear 1,ARKANSAS NUCLEAR ONE
1,Arkansas Nuclear 2,ARKANSAS NUCLEAR ONE
2,Beaver Valley 1,BEAVER VALLEY
3,Beaver Valley 2,BEAVER VALLEY
4,Braidwood 1,BRAIDWOOD


In [92]:
#make this a dictionary 
site_dict = convert.set_index('Plant')['Site'].to_dict()

In [93]:
#apply the unit-site conversion dictionary site_dict to the am_dataframe to get a column by site
am['Site']= am['Unit'].map(site_dict)

In [94]:
am.head()

,Unit,Quarter,Year,Column #,Site
0,Arkansas Nuclear 1,2006Q1,2006,1,ARKANSAS NUCLEAR ONE
1,Arkansas Nuclear 1,2006Q2,2006,1,ARKANSAS NUCLEAR ONE
2,Arkansas Nuclear 1,2006Q3,2006,1,ARKANSAS NUCLEAR ONE
3,Arkansas Nuclear 1,2006Q4,2006,1,ARKANSAS NUCLEAR ONE
4,Arkansas Nuclear 1,2007Q1,2007,1,ARKANSAS NUCLEAR ONE


In [95]:
#simplify the dataframe to be by site and eliminate unit
am = am[["Site", "Quarter", "Year", "Column #"]]

In [96]:
am.head()

,Site,Quarter,Year,Column #
0,ARKANSAS NUCLEAR ONE,2006Q1,2006,1
1,ARKANSAS NUCLEAR ONE,2006Q2,2006,1
2,ARKANSAS NUCLEAR ONE,2006Q3,2006,1
3,ARKANSAS NUCLEAR ONE,2006Q4,2006,1
4,ARKANSAS NUCLEAR ONE,2007Q1,2007,1


In [97]:
#find the maximum action matrix column at a site in a given year
am_final = am.groupby(by=['Site', 'Year']).max()[["Column #"]].reset_index()

In [98]:
am_final.head()

,Site,Year,Column #
0,ARKANSAS NUCLEAR ONE,2006,1
1,ARKANSAS NUCLEAR ONE,2007,1
2,ARKANSAS NUCLEAR ONE,2008,1
3,ARKANSAS NUCLEAR ONE,2009,2
4,ARKANSAS NUCLEAR ONE,2010,1


In [104]:
#verify same number of rows as in data
am_final.shape

(883, 3)

In [107]:
df1 = am_final.sort_values(by =['Site', 'Year'])

In [108]:
df1.to_excel('am_data.xlsx')

In [109]:
df1.shape

(883, 3)

In [110]:
#open the data.xlsx file and append the action matrix column # 
data = pd.read_excel("data.xlsx")

In [112]:
data.head(2)

,Unnamed: 0,SiteName,Year,ENCount,ScramCount
0,0,ARKANSAS NUCLEAR ONE,2006,2,0
1,1,ARKANSAS NUCLEAR ONE,2007,2,1


In [113]:
#eliminate unnecessary index column from data
data = data[["SiteName", "Year", "ENCount", "ScramCount"]]
data.head(2)

,SiteName,Year,ENCount,ScramCount
0,ARKANSAS NUCLEAR ONE,2006,2,0
1,ARKANSAS NUCLEAR ONE,2007,2,1


In [120]:
new_data = pd.merge(data, am_final, how='left', left_index=True, right_index=True, suffixes=("", '_am'))

In [121]:
new_data.head(5)

,SiteName,Year,ENCount,ScramCount,Site,Year_am,Column #
0,ARKANSAS NUCLEAR ONE,2006,2,0,ARKANSAS NUCLEAR ONE,2006,1
1,ARKANSAS NUCLEAR ONE,2007,2,1,ARKANSAS NUCLEAR ONE,2007,1
2,ARKANSAS NUCLEAR ONE,2008,5,2,ARKANSAS NUCLEAR ONE,2008,1
3,ARKANSAS NUCLEAR ONE,2009,12,5,ARKANSAS NUCLEAR ONE,2009,2
4,ARKANSAS NUCLEAR ONE,2010,4,2,ARKANSAS NUCLEAR ONE,2010,1


In [118]:
new_data[(new_data["SiteName"] != new_data["Site"])]

,SiteName,Year_data,ENCount,ScramCount,Site,Year_am,Column #


In [122]:
data2 = new_data[["Site", "Year", "ENCount", "ScramCount", "Column #"]]

In [123]:
data2.to_excel("EnsAndAm.xlsx")